# Imports

In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from typing import Set, Dict, Callable

# Defining Constants

In [2]:
# Define constants
CWD = os.path.abspath('')
CWD = Path(CWD)
CRISPR_DATA = CWD/'CRISPR_Data'
DEPMAP = CWD / 'DepMap'

SOD_LBL = 'PSOD_LT_FDR.05'
SOD_CUTOFFS = (.05, .1)
IPE_CUTOFFS = (.25, -.1)
IPE_LBL = 'PIPE_GT_.25FDR_-.1LFC'
COMM_ESS = 'Common_Essential'
MAIN_OUT = 'PEL_All_Data.csv'
SODS_OUT = 'PSODs.csv'
IPES_OUT = 'PIPEs.csv'
LRT_OUT = 'NormLRT.csv'

EXPORT_SETTINGS = {
    'sep': ',',
    'header': True,
    'index': True
}

# Defining Functions

In [3]:
def import_and_merge(dir: Path = CRISPR_DATA, suffix: str = '.csv',
                     sep: str = ',') -> pd.DataFrame:
    """
    Imports MAGeCK data from multiple screens into one dataframe

    Parameters
    ----------
    dir : Path, optional
        Directory containing MAGeCK data, by default CRISPR_DATA
        file names should be meaningful (i.e. cell line/condition names)
    suffix : str, optional
        The file suffix of target MAGeCK Output Files, by default '.csv'
    sep : str, optional
       Field separator for MAGeCK data, by default ','

    Returns
    -------
    pd.DataFrame
        A dataframe with rows of genes and columns of MAGeCK depletion
        metrics (NegFDR/LFC)
    """
    df_dict ={}
    for f in list(dir.glob(f'*{suffix}')):
        cell_line = f.name.strip(suffix)
        df_dict[cell_line] = pd.read_csv(f, sep=',', header=0, index_col=0)

    for cell_line, df in df_dict.items():
        new_cols = {
            'neg|fdr': f'{cell_line}|NegFDR',
            'neg|lfc': f'{cell_line}|NegLFC'
            }
        df.rename(columns=new_cols, inplace=True)
        df_dict[cell_line] = df[[f'{cell_line}|NegFDR', f'{cell_line}|NegLFC']]
    dfs_list = [v for v in df_dict.values()]
    merged_df = pd.concat(dfs_list,axis=1,join="inner")

    return merged_df

# Should be run on MAGeCK data before anything else
# Typically twice (Achilles+SCORE combined vs Achilles-only genes)
def check_comm_ess(df: pd.DataFrame, prefix: str, common_essentials: Set[str],
                   included_guides: Set[str]):
    """
    Checks whether each gene is included in a subset of DepMap.
    
    Modifies in-place, does not return.

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe, containing MAGeCK screen data
    prefix : str
        A label to signify this DepMap subset (i.e. Achilles-only)
    common_essentials : Set[str]
        A set of guides designated as common essentials in a subset of DepMap.
    included_guides : Set[str]
        The set of guides included in this subset of DepMap.
    """
    comm_ess = f'{prefix}_Comm_Ess'
    df[comm_ess] = np.nan
    df[f'{prefix}_hasGuide'] = df.index.isin(included_guides)
    has_guide = df[f'{prefix}_hasGuide']
    df.loc[has_guide, comm_ess] = df[has_guide].index.isin(common_essentials)

# Run after labeling common essentials from one or more datasets
def if_not_na_else(df: pd.DataFrame, new_col: str, if_not_na_col: str,
                   else_col: str):
    """
    Creates a new column C in a "if A != NA, B" manner.

    Modifies in-place, does not return.

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe, containing MAGeCK screen data
    new_col : str
        Name of the new column (i.e. col C)
    if_not_na_col : str
        Name of the default column to use (if not NA) (i.e. col A)
    else_col : str
        Name of the backup column to use (i.e. col B)
        False if there is no backup (propagates NAs from col A)
    """
    non_null = df[if_not_na_col].isnull()
    if else_col:
        vals = np.where(non_null, df[else_col], df[if_not_na_col])
    else:
        df[new_col] = df[if_not_na_col]
    df[new_col] = vals
    
# Must before labeling SODs
def lbl_highly_selective(df: pd.DataFrame,
                         highly_selectives: Set[str]):
    """
    Labels genes as highly selective or not.

    Modifies in-place, does not return.

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe, containing MAGeCK screen data
    highly_selectives : Set[str]
        A set or other container to check for membership.
    """
    df['Highly_Selective'] = False
    df.loc[df.index.isin(highly_selectives),'Highly_Selective'] = True

# Must run lbl_highly_selective first
def lbl_cohort_sods(df: pd.DataFrame, output_col: str, fdr_thresh: float = .05,
                    lfc_thresh: float = 0, metric: str = 'median'):
    """
    Labels genes as a cohort-specific oncogenic dependency (SOD) in the
    non-DepMap cohort, based on thresholded filtering.

    Modifies in-place, does not return.

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe, containing MAGeCK screen data
    output_col : str
        Name for output column
    fdr_thresh : float, optional
        Maximum FDR to allow, by default .05
    lfc_thresh : float, optional
        Maximum LFC to allow, by default 0
    metric : str, optional
        How to aggregate cohort data (mean or median), by default 'median'
    """
    fdrs = [i for i in df.columns if 'NegFDR' in i]
    lfcs = [i for i in df.columns if 'NegLFC' in i]
    if metric == 'median':
        df['Median|NegFDR'] = np.median(df[fdrs], axis=1)
        df['Median|NegLFC'] = np.median(df[lfcs], axis=1)
        fdr_col = 'Median|NegFDR'
        lfc_col = 'Median|NegLFC'
    elif metric == 'mean':
        df['Mean|NegFDR'] = np.mean(df[fdrs], axis=1)
        df['Mean|NegLFC'] = np.mean(df[lfcs], axis=1)
        fdr_col = 'Mean|NegFDR'
        lfc_col = 'Mean|NegLFC'
    not_comm_ess = df['Common_Essential'] == False
    highly_specific = df['Highly_Selective'] == True
    uncommon = np.logical_or(not_comm_ess, highly_specific)
    meets_thresh = (df[fdr_col] <= fdr_thresh) & (df[lfc_col] <= lfc_thresh)
    df[output_col] = np.where(uncommon & meets_thresh, True, False)

def lbl_cohort_ipes(df: pd.DataFrame, output_col: str, fdr_thresh: float = .25,
                    lfc_thresh: float = 0, metric: str = 'median') -> None:
    """
    Labels genes as a cohort-insensitive (SOD) in the non-DepMap
    cohort, based on filtering.

    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe, containing MAGeCK screen data
    output_col : str
        Name for output column
    fdr_thresh : float, optional
        Minimum FDR to allow, by default .25
    lfc_thresh : float, optional
        Minimum LFC to allow, by default 0
    metric : str, optional
        How to aggregate cohort data (mean or median), by default 'median'
    """
    if metric == 'median':
        fdr_col = 'Median|NegFDR'
        lfc_col = 'Median|NegLFC'
    elif metric == 'mean':
        fdr_col = 'Mean|NegFDR'
        lfc_col = 'Mean|NegLFC'
    comm_ess = df['Common_Essential'] == True
    meets_thresh = (df[fdr_col] >= fdr_thresh) & (df[lfc_col] >= lfc_thresh)
    df[output_col] = np.where(comm_ess & meets_thresh, True, False)

# Used to return a df of NormLRT values; optional.
def run_r_func(func: Callable[[pd.DataFrame, dict], pd.DataFrame],
               ge_tables: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    Calculates NormLRT values from CHRONOS data (assumes no overlap in genes)

    Parameters
    ----------
    func : Callable[[pd.DataFrame, dict], pd.DataFrame]
        An R function (intended to be LRT_test from DepMap consortium)
        Must be defined properly with RPy2 or equivalent. See norm_lrt.py
    ge_tables : Dict[str, pd.DataFrame]
        A dictionary of CHRONOS dataframes (rows screens/columns genes).
        Dictionary key is used for progress bar.
        It is assumed that dataframes have already been filtered to avoid
        overlap between genes.
    Returns
    -------
    pd.DataFrame
        A dataframe, with genes as rows and a single 'NormLRT' when used with
        the LRT_test() function from DepMap consortium.
    """
    lrts = []
    for k,v in ge_tables.items():
        lrts.append(func(v, k))
    lrt_df = pd.concat(lrts, axis=0, join='inner', verify_integrity=True)
    return lrt_df

# Loading and examining data

In [4]:
#### Load in DepMap data
print("Loading and processing DepMap data")

# CRISPR (Achilles + Score)
crispr_comm_essential = pd.read_csv(DEPMAP/'CRISPR_common_essentials.csv')
crispr_comm_essential = crispr_comm_essential['gene'].tolist()
crispr_comm_essential = [g.split(' ')[0] for g in crispr_comm_essential]
crispr_ge = pd.read_csv(DEPMAP/'CRISPR_gene_effect.csv', index_col=0)
renames = {c:c.split(' ')[0] for c in crispr_ge.columns}
crispr_ge.rename(columns=renames,inplace=True)
crispr_guides = set(crispr_ge.columns)

# Achilles(Achilles)
ach_comm_essential = pd.read_csv(DEPMAP/'Achilles_common_essentials.csv')
ach_comm_essential = ach_comm_essential['gene'].tolist()
ach_comm_essential = [g.split(' ')[0] for g in ach_comm_essential]
ach_ge = pd.read_csv(DEPMAP/'Achilles_gene_effect.csv', index_col=0)
renames = {c:c.split(' ')[0] for c in ach_ge.columns}
ach_ge.rename(columns=renames,inplace=True)
ach_guides = set(ach_ge.columns)
depmap_guides = crispr_guides.union(ach_guides)

#Subset Achille's GE matrix to exclude CRISPR (Achilles+Score) genes.
ach_ge = ach_ge[[c for c in ach_ge.columns if c not in crispr_guides]]

#Load DepMap list of highly-selective genes
hs_genes = pd.read_csv(DEPMAP/'DepMap_Selective_Genes.csv')
hs_genes = set(hs_genes['gene'].values)

Loading and processing DepMap data


In [5]:
ach_only_comm_essential = set([g for g in ach_comm_essential if g not in crispr_guides])
all_comm_essentials = ach_only_comm_essential.union(crispr_comm_essential)
hs_comm_essentials = all_comm_essentials.intersection(hs_genes)

print(f'There are {len(all_comm_essentials)} common essential genes in this DepMap release.')
print(f'There are {len(hs_genes)} highly-selective genes.')
print(f'There are {len(hs_comm_essentials)} highly-selective, common essential genes.')

There are 1982 common essential genes in this DepMap release.
There are 3741 highly-selective genes.
There are 86 highly-selective, common essential genes.


In [6]:
print("Loading and processing MAGeCK data")
merged_df = import_and_merge()
merged_df.head()

Loading and processing MAGeCK data


,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,VG1|NegFDR,VG1|NegLFC,BC5|NegFDR,BC5|NegLFC,BCBL1|NegFDR,BCBL1|NegLFC
id,,,,,,,,,,,,,,,,
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,0.000990,-2.9889,0.000309,-3.6189,0.000464,-5.1584
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,0.013539,-1.8489,0.005099,-2.5673,0.000275,-4.4837
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,0.005215,-2.5877,0.000309,-3.1136,0.000275,-4.6341
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,0.001433,-1.8017,0.000309,-2.1269,0.001206,-3.6375
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,0.003753,-1.9387,0.000309,-2.1948,0.000275,-4.9310


In [7]:
cohort_genes = set(merged_df.index)
overlap_comm = all_comm_essentials.intersection(cohort_genes)
overlap_hs = hs_genes.intersection(cohort_genes)
overlap_hs_comm = hs_comm_essentials.intersection(cohort_genes)

print(f'{len(overlap_comm)} of {len(all_comm_essentials)} DepMap common essentials are in the cohort library.')
print(f'{len(overlap_hs)} of {len(hs_genes)} DepMap highly-selective genes are in the cohort library')
print(f'{len(overlap_hs_comm)} of {len(hs_comm_essentials)} DepMap highly-selective, common essentials genes are in the cohort library')

1823 of 1982 DepMap common essentials are in the cohort library.
3558 of 3741 DepMap highly-selective genes are in the cohort library
80 of 86 DepMap highly-selective, common essentials genes are in the cohort library


In [8]:
# Check if highly LRTs have been calculated
LRT_FILE = CWD / LRT_OUT

if LRT_FILE .is_file():
    print('Loading pre-calculated LRT values')
    lrt_initialized = True
    lrt_df = pd.read_csv(LRT_FILE, sep=',', index_col=0)
else:
    from norm_lrt import lrt_test

Loading pre-calculated LRT values


# Analysis/Filtering

In [9]:
print("Beginning analysis")
check_comm_ess(merged_df, 'Ach+SCORE', crispr_comm_essential, crispr_guides)
merged_df.head()

Beginning analysis


,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,VG1|NegFDR,VG1|NegLFC,BC5|NegFDR,BC5|NegLFC,BCBL1|NegFDR,BCBL1|NegLFC,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide
id,,,,,,,,,,,,,,,,,,
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,0.000990,-2.9889,0.000309,-3.6189,0.000464,-5.1584,True,True
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,0.013539,-1.8489,0.005099,-2.5673,0.000275,-4.4837,True,True
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,0.005215,-2.5877,0.000309,-3.1136,0.000275,-4.6341,True,True
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,0.001433,-1.8017,0.000309,-2.1269,0.001206,-3.6375,True,True
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,0.003753,-1.9387,0.000309,-2.1948,0.000275,-4.9310,False,True


In [10]:
check_comm_ess(merged_df, 'Ach', ach_comm_essential, ach_guides)
merged_df.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,VG1|NegFDR,VG1|NegLFC,BC5|NegFDR,BC5|NegLFC,BCBL1|NegFDR,BCBL1|NegLFC,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide
id,,,,,,,,,,,,,,,,,,,,
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,0.000990,-2.9889,0.000309,-3.6189,0.000464,-5.1584,True,True,True,True
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,0.013539,-1.8489,0.005099,-2.5673,0.000275,-4.4837,True,True,True,True
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,0.005215,-2.5877,0.000309,-3.1136,0.000275,-4.6341,True,True,True,True
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,0.001433,-1.8017,0.000309,-2.1269,0.001206,-3.6375,True,True,True,True
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,0.003753,-1.9387,0.000309,-2.1948,0.000275,-4.9310,False,True,False,True


In [11]:
if_not_na_else(merged_df, COMM_ESS, 'Ach+SCORE_Comm_Ess', 'Ach_Comm_Ess')
merged_df.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,VG1|NegLFC,BC5|NegFDR,BC5|NegLFC,BCBL1|NegFDR,BCBL1|NegLFC,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential
id,,,,,,,,,,,,,,,,,,,,,
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,...,-2.9889,0.000309,-3.6189,0.000464,-5.1584,True,True,True,True,True
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,...,-1.8489,0.005099,-2.5673,0.000275,-4.4837,True,True,True,True,True
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,...,-2.5877,0.000309,-3.1136,0.000275,-4.6341,True,True,True,True,True
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,...,-1.8017,0.000309,-2.1269,0.001206,-3.6375,True,True,True,True,True
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,...,-1.9387,0.000309,-2.1948,0.000275,-4.9310,False,True,False,True,False


In [12]:
if not(lrt_initialized):
    lrt_df = run_r_func(_lrt_test, {'Achilles+Score': crispr_ge, 'Achilles-only': ach_ge})

lrt_df = lrt_df[lrt_df.index.isin(merged_df.index)]
lrt_df.head()

,Skew_LRT
HSPA5,530.854411
PRMT1,1.927878
PSMB4,281.501610
FAU,665.849975
CCND2,1366.649183


In [13]:
merged_df = pd.concat([merged_df, lrt_df], axis=1, join='outer')
merged_df.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,BC5|NegFDR,BC5|NegLFC,BCBL1|NegFDR,BCBL1|NegLFC,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,...,0.000309,-3.6189,0.000464,-5.1584,True,True,True,True,True,530.854411
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,...,0.005099,-2.5673,0.000275,-4.4837,True,True,True,True,True,1.927878
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,...,0.000309,-3.1136,0.000275,-4.6341,True,True,True,True,True,281.501610
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,...,0.000309,-2.1269,0.001206,-3.6375,True,True,True,True,True,665.849975
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,...,0.000309,-2.1948,0.000275,-4.9310,False,True,False,True,False,1366.649183


In [14]:
lbl_highly_selective(merged_df, hs_genes)
merged_df.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,BC5|NegLFC,BCBL1|NegFDR,BCBL1|NegLFC,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Highly_Selective
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,...,-3.6189,0.000464,-5.1584,True,True,True,True,True,530.854411,False
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,...,-2.5673,0.000275,-4.4837,True,True,True,True,True,1.927878,False
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,...,-3.1136,0.000275,-4.6341,True,True,True,True,True,281.501610,False
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,...,-2.1269,0.001206,-3.6375,True,True,True,True,True,665.849975,False
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,...,-2.1948,0.000275,-4.9310,False,True,False,True,False,1366.649183,True


In [15]:
lbl_cohort_sods(merged_df, SOD_LBL, *SOD_CUTOFFS)
merged_df.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Highly_Selective,Median|NegFDR,Median|NegLFC,PSOD_LT_FDR.05
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,...,True,True,True,True,True,530.854411,False,0.000387,-4.02200,False
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,...,True,True,True,True,True,1.927878,False,0.000826,-3.31435,False
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,...,True,True,True,True,True,281.501610,False,0.000567,-3.48895,False
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,...,True,True,True,True,True,665.849975,False,0.000758,-2.89800,False
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,...,False,True,False,True,False,1366.649183,True,0.000550,-3.16525,True


In [16]:
lbl_cohort_ipes(merged_df, IPE_LBL, *IPE_CUTOFFS)
merged_df.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Highly_Selective,Median|NegFDR,Median|NegLFC,PSOD_LT_FDR.05,PIPE_GT_.25FDR_-.1LFC
HSPA5,0.000248,-5.2452,0.000291,-3.9107,0.000190,-4.13330,0.001089,-4.3797,0.000550,-3.3787,...,True,True,True,True,530.854411,False,0.000387,-4.02200,False,False
PRMT1,0.000248,-3.3989,0.000291,-5.2478,0.000190,-3.94850,0.001518,-3.2298,0.001361,-2.8519,...,True,True,True,True,1.927878,False,0.000826,-3.31435,False,False
PSMB4,0.000248,-4.9707,0.002304,-2.9662,0.000190,-4.05290,0.000825,-3.8643,0.001867,-2.2212,...,True,True,True,True,281.501610,False,0.000567,-3.48895,False,False
FAU,0.000248,-3.7419,0.000291,-2.9393,0.000190,-3.68490,0.007031,-2.8567,0.001361,-2.5343,...,True,True,True,True,665.849975,False,0.000758,-2.89800,False,False
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,...,True,False,True,False,1366.649183,True,0.000550,-3.16525,True,False


# Export results and summarize

In [17]:
print("Exporting reports")

merged_df.to_csv(CWD/MAIN_OUT, **EXPORT_SETTINGS)

norm_lrt_df = merged_df['Skew_LRT']
norm_lrt_df.to_csv(CWD / LRT_FILE, sep=',', index=True)

Exporting reports


In [18]:
sods = merged_df.loc[merged_df[SOD_LBL] == True]
sods = sods.drop(columns=[IPE_LBL])
has_guides = sods.index.isin(depmap_guides)
high_conf = sods.loc[has_guides]
high_conf = high_conf.loc[sods[COMM_ESS] == False]
high_conf.head()
comm_ess = sods[COMM_ESS] == True

In [19]:
print(f'{len(high_conf)} high-confidence SODs')
is_hs = high_conf['Highly_Selective'] == True
print(f'{len(high_conf[is_hs])} high-confidence, highly-selective SODs')
high_conf.head()

71 high-confidence SODs
25 high-confidence, highly-selective SODs


,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Highly_Selective,Median|NegFDR,Median|NegLFC,PSOD_LT_FDR.05
CCND2,0.000248,-4.1234,0.001310,-3.5028,0.010392,-0.92289,0.000550,-3.9242,0.000550,-2.8277,...,False,True,False,True,False,1366.649183,True,0.000550,-3.16525,True
MTHFD1,0.000248,-5.0115,0.000675,-3.7987,0.000190,-3.27830,0.017473,-1.7028,0.004200,-2.6842,...,False,True,False,True,False,269.440914,True,0.002437,-2.98125,True
UQCRC1,0.000594,-2.8992,0.001396,-2.2440,0.000190,-2.83510,0.003025,-1.8024,0.001934,-1.5414,...,False,True,False,True,False,1.944162,False,0.002337,-2.02320,True
RPUSD4,0.001480,-2.6500,0.051257,-1.6624,0.004154,-2.29840,0.179773,-1.3459,0.001762,-2.1957,...,False,True,False,True,False,59.636740,False,0.027705,-1.92905,True
ALAS1,0.001861,-3.2143,0.000291,-3.3803,0.001001,-3.50400,0.005582,-2.6185,0.003854,-2.2437,...,False,True,False,True,False,343.595274,True,0.004718,-2.91640,True


In [20]:
high_conf.groupby(['Highly_Selective'])['Highly_Selective'].agg('count')

Highly_Selective
False    46
True     25
Name: Highly_Selective, dtype: int64

In [21]:
putative = sods[np.logical_or(comm_ess, ~has_guides)].copy()
    
putative['Reason'] = ''
putative.loc[comm_ess, 'Reason'] = 'Pan-Essential'
putative.loc[~comm_ess, 'Reason'] = 'No Data'

In [22]:
print(f'{len(putative)} putative SODs.')
putative.head()

16 putative SODs.


,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Highly_Selective,Median|NegFDR,Median|NegLFC,PSOD_LT_FDR.05,Reason
AK2,0.001480,-3.7968,0.001195,-4.2801,0.005926,-3.9063,0.016431,-2.0582,0.007013,-2.6578,...,True,False,True,True,170.094067,True,0.006469,-3.22730,True,Pan-Essential
PPCS,0.002032,-3.5453,0.050041,-1.9219,0.001598,-2.9097,0.105645,-1.3718,0.007953,-1.7925,...,True,False,True,True,191.117948,True,0.028997,-1.85720,True,Pan-Essential
EIF3CL,0.002986,-2.2256,0.009682,-2.1084,0.005416,-3.1412,0.035231,-2.5191,0.005718,-2.3804,...,False,NaN,False,NaN,NaN,True,0.008918,-2.16700,True,No Data
GLRX5,0.003340,-4.2150,0.009682,-2.6829,0.005582,-2.9736,0.002382,-3.0444,0.002932,-2.3419,...,True,True,True,True,107.100441,True,0.005932,-2.82825,True,Pan-Essential
UROD,0.004240,-3.9179,0.035057,-2.6282,0.009253,-3.0888,0.035470,-1.9369,0.005718,-1.9448,...,True,True,True,True,121.006472,True,0.022155,-2.28650,True,Pan-Essential


In [23]:
putative.groupby(['Reason'])['Reason'].agg('count')

Reason
No Data           5
Pan-Essential    11
Name: Reason, dtype: int64

In [24]:
putative.groupby(['Highly_Selective']).agg('count')

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Median|NegFDR,Median|NegLFC,PSOD_LT_FDR.05,Reason
Highly_Selective,,,,,,,,,,,,,,,,,,,,,
True,16,16,16,16,16,16,16,16,16,16,...,11,16,11,16,11,11,16,16,16,16


In [25]:
high_conf_out = 'HighConf_'+SODS_OUT
putative_out = 'Putative_'+SODS_OUT
high_conf.to_csv(CWD/high_conf_out, **EXPORT_SETTINGS)
putative.to_csv(CWD/putative_out, **EXPORT_SETTINGS)

In [26]:
ipes = merged_df[merged_df[IPE_LBL] == True]
ipes = ipes.drop(columns=[SOD_LBL])
ipes.to_csv(CWD/IPES_OUT, **EXPORT_SETTINGS)

In [27]:
print(f'{len(ipes)} IPEs')
is_hs = ipes['Highly_Selective'] == True
print(f'{len(ipes[is_hs])} highly-selective IPEs')

120 IPEs
13 highly-selective IPEs


In [28]:
ipes.head()

,JSC1|NegFDR,JSC1|NegLFC,BC2|NegFDR,BC2|NegLFC,BC3|NegFDR,BC3|NegLFC,APK1|NegFDR,APK1|NegLFC,BC1|NegFDR,BC1|NegLFC,...,Ach+SCORE_Comm_Ess,Ach+SCORE_hasGuide,Ach_Comm_Ess,Ach_hasGuide,Common_Essential,Skew_LRT,Highly_Selective,Median|NegFDR,Median|NegLFC,PIPE_GT_.25FDR_-.1LFC
C1D,0.624270,-0.696650,1.000000,0.28269,1.000000,0.214670,0.999999,0.513280,0.999998,-0.359380,...,True,True,True,True,True,49.248839,False,0.999999,0.055980,True
SPRR2G,0.922098,-0.357780,1.000000,0.14357,1.000000,0.098737,0.999999,-0.068963,0.999998,-0.114690,...,True,True,False,True,True,40.734650,False,0.999999,0.014887,True
ZNF593,0.999998,-0.079231,1.000000,0.18329,1.000000,-0.132440,0.493998,-0.558510,0.999998,0.070150,...,True,True,False,True,True,43.145165,False,0.999999,-0.012880,True
FBXO5,0.999998,-0.693480,1.000000,0.12215,0.267025,-1.101900,0.999999,0.030629,0.999998,0.015687,...,True,True,True,True,True,17.782805,False,0.999999,0.034227,True
OIP5,0.999998,0.102010,0.410011,-1.51940,1.000000,0.065290,0.999999,-0.122590,0.980162,-0.053040,...,True,True,True,True,True,147.369579,False,0.990080,-0.087815,True


In [29]:
print("Complete!")

Complete!
